# Process Rift Inversion Model

In [1]:
# Imports
import os
import shutil

import cv2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvista as pv

pv.start_xvfb()

from tqdm import tqdm
from joblib import Parallel,delayed

from gdtchron import calculate,plot,aft

In [ ]:
# Get full output files from local source
part_path = '/home/vaseylab/glaucophane/fastscape_testing/060524_rif/output_ri_rift/particles'
files = [os.path.join(part_path,file) for file in os.listdir(part_path) if file.endswith('.pvtu')]
files.sort()

In [ ]:
# Define directory for clipped meshes
clip_dir = 'ri_clipped'

In [ ]:
# Isolate temperature and particle id and trim
bounds =[200e3,800e3,550e3,620e3,0,0] # km (X,Y,Z)
os.makedirs(clip_dir,exist_ok=True)

def trim_file(file,bounds,clip_dir):
    basename = os.path.basename(file)[:-5] + '.vtu'
    filename = os.path.join(clip_dir,basename)
    
    if os.path.exists(filename):
        return
    
    mesh=pv.read(file)
    temps = mesh['T']
    ids = mesh['id']
    mesh.point_data.clear()
    mesh['T'] = temps
    mesh['id'] = ids
    clipped=mesh.clip_box(bounds=bounds,invert=False)
    clipped.save(filename)
    return

output = Parallel(n_jobs=os.cpu_count()/2)(delayed(trim_file)(file,bounds,clip_dir) for file in tqdm(files,position=0))

In [ ]:
# Get path to new files
files = [os.path.join(clip_dir,file) for file in os.listdir(clip_dir) if file.endswith('.vtu')]
files.sort()

In [ ]:
# Add AHe ages and create new meshes.
calculate.tchron_vtk_parallel(files,'AHe',1e5,
                                model_inputs=(100,100,50),
                                model_fn=calculate.particle_He_profile,
                                batch_size=1000,
                                processes=os.cpu_count()/2,
                                overwrite=False)

In [ ]:
# Set path to reuse files from AHe
new_path = 'meshes_tchron'
files = [os.path.join(new_path,file) for file in os.listdir(new_path) if file.endswith('.vtu')]
files.sort()

In [ ]:
# Add AFT
calculate.tchron_vtk_parallel(files,'AFT',1e5,
                                model_inputs=(aft.Ketcham_99_FC, 1.75),
                                model_fn=calculate.particle_FT_annealing,
                                batch_size=1000,
                                processes=os.cpu_count()/2,
                                overwrite=False)

In [ ]:
# Add ZHe
calculate.tchron_vtk_parallel(files,'ZHe',1e5,
                                model_inputs=(100,100,50),
                                model_fn=calculate.particle_He_profile,
                                batch_size=1000,
                                processes=os.cpu_count()/2,
                                overwrite=False)

In [ ]:
# Run AHe without interpolation
# Add AHe ages and create new meshes.
calculate.tchron_vtk_parallel(files,'AHe',1e5,
                                model_inputs=(100,100,50),
                                model_fn=calculate.particle_He_profile,
                                batch_size=1000,
                                processes=os.cpu_count()/2,
                                overwrite=False,
                                file_prefix='meshes_ahe_nointerp',
                                interpolate_vals=False
                                )